## tabelog scraper
#### 指定したエリア周辺のレストランの

In [17]:
import requests
from bs4 import BeautifulSoup
import time
import re
import csv

class tabelog_scraper:
    def __init__(self):
        self.base_url = None
        self.response = None
        self.num_pages = 1
    
    def get_response(self, url):
        self.response = requests.get(url,timeout=(12.0, 12.0))
        
    def set_num_pages(self):
        # 読み込むページ数をセット
        self.get_response(self.base_url)
        soup = BeautifulSoup(self.response.text, 'lxml')
        value = soup.find_all("a",class_="c-pagination__num" )
        num_pages = int(re.findall("([0-9]+)",str(value[-1]))[-1])
        print(num_pages)
        self.num_pages = num_pages
        
    def html_extractor(self, value):
        # タグを除いたテキストを取得
        return re.findall(r">(.*?)<",value)[0]
    
    def souping(self):
        # [店名、レート、レビュー数]をリスト型で返す
        soup = BeautifulSoup(self.response.content, 'html.parser')

        rc_list = []
        for rc_div in soup.find_all('div', class_='list-rst__wrap js-open-new-window'):
            rc_name = rc_div.find('a', class_='list-rst__rst-name-target cpy-rst-name').text
            rc_score = rc_div.find('span', class_='c-rating__val c-rating__val--strong list-rst__rating-val')
            if rc_score is None:
                rc_score = -1.
            else:
                rc_score = float(rc_score.text)
            rc_review_num = rc_div.find('em', class_='list-rst__rvw-count-num cpy-review-count').text
            rc_list.append([rc_name, rc_score, rc_review_num])
        return rc_list


    def scraper(self, file_name):
        # 店名、レート、レビュー数をcsvファイルに記録
        with open(file_name, 'w') as file:
            writer = csv.writer(file)
            header = ['name','rating','num_votes']
            writer.writerow(header)
            url = self.base_url
            for page_index in range(self.num_pages):
                if page_index > 0:
                    url = self.base_url + "rstLst/" + str(page_index + 1)
                self.get_response(url)
                restaurant_data = self.souping()
                for row in restaurant_data:
                    writer.writerow(row)

            

In [18]:
scr = tabelog_scraper()
#今回は幕張エリアのレストランをスクレイピング
scr.base_url = "https://tabelog.com/chiba/A1201/A120102/"
scr.set_num_pages()
scr.num_pages
scr.scraper('t_ratings.csv')

30
